---

# CSCI 3202, Fall 2022
# Homework 2: MDP & Reinforcement Learning
# Due: Friday September 9, 2022 at 6:00 PM

<br> 

### Your name: Ty VanEssen

<br> 

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict

# added packages
import heapq
from matplotlib import colors



---

Consider a **cube** state space defined by $0 \le x, y, z \le L$. Suppose you are piloting/programming a drone to learn how to land on a platform at the center of the $z=0$ surface (the bottom). Some assumptions:
* In this discrete world, if the drone is at $(x,y,z)$ it means that the box is centered at $(x,y,z)$. There are boxes (states) centered at $(x,y,z)$ for all $0 \le x,y,z \le L$. Each state is a 1 unit cube. 
* In this world, $L$ is always an even value.
* All of the states with $z=0$ are terminal states.
* The state at the center of the bottom of the cubic state space is the landing pad. For example, when $L=4$, the landing pad is at $(x,y,z) = (2,2,0)$.
* All terminal states ***except*** the landing pad have a reward of -1. The landing pad has a reward of +1.
* All non-terminal states have a living reward of -0.01.
* The drone takes up exactly 1 cubic unit, and begins in a random non-terminal state.
* The available actions in non-terminal states include moving exactly 1 unit Up (+z), Down (-z), North (+y), South (-y), East (+x) or West (-x). In a terminal state, the training episode should end.

#### Part A
How many states would be in the discrete state space if $L=2$? Explain your reasoning.

3^3 = 27 states.
Possible sub-states:
- Z: 3. (0,1,2)
- X: 3
- Y: 3

#### Part B
Write a class `MDPLanding` to represent the Markov decision process for this drone. Include methods for:
1. `actions(state)`, which should return a list of all actions available from the given state
2. `reward(state)`, which should return the reward for the given state
3. `result(state, action)`, which should return the resulting state of doing the given action in the given state

and attributes for:
1. `states`, a list of all the states in the state space, where each state is represented as an $(x,y,z)$ tuple
2. `terminal_states`, a dictionary where keys are the terminal state tuples and the values are the rewards associated with those terminal states
3. `default_reward`, a scalar for the reward associated with non-terminal states
4. `all_actions`, a list of all possible actions (Up, Down, North, South, East, West)
5. `discount`, the discount factor (use $\gamma = 0.999$ for this entire problem)

How you feed arguments/information into the class constructor is up to you.

Note that actions are *deterministic* here.  The drone does not need to include transition probabilities for outcomes of particular actions. What the drone does need to learn, however, is where the landing pad is, and how to get there from any initial state.

Before moving on to Part C, we recommend that you test that your MDPLanding code is set up correctly. Write unit tests that display the actions for a given state, rewards, results, etc. This will help you identify errors in your implementation and save you a lot of debugging time later.

In [11]:
# Solution:
from typing import Tuple, Type


State: Type[Tuple[int, int, int]] = (int, int, int)

class MDPLanding:
    default_reward = -0.01
    all_actions = ["Up","Down","North","South","East","West"]
    discount = 0.999

    def __init__(self, L : int):
        self.L = L
        self.states = [(x,y,z) for x in range(L + 1) for y in range(L + 1) for z in range(L + 1)]
        self.terminal_states = {(L/2, L/2, 0): 1}
        [ self.terminal_states.setdefault((x,y,0), -1) for x in range(L + 1) for y in range(L + 1) ]

    def actions(self, state):
        possible_actions = MDPLanding.all_actions.copy()
        x = state[0]
        y = state[1]
        z = state[2]

        if x >= self.L:
            possible_actions.remove("East")
        if x <= 0:
            possible_actions.remove("West")
        if y >= self.L:
            possible_actions.remove("North")
        if y <= 0:
            possible_actions.remove("South")
        if z >= self.L:
            possible_actions.remove("Up")
        if z <= 0:
            possible_actions.remove("Down")

        return possible_actions
    
    def reward(self, state: State):
        if state in self.terminal_states:
            return self.terminal_states[state]
        else:
            return MDPLanding.default_reward

    def result(self, state: Tuple[int, int, int], action : all_actions) -> Tuple[int, int, int]:
        match action:
            case "Up":
                return (*state[0:2], state[2] + 1)
            case "Down":
                return (*state[0:2], state[2] - 1)
            case "North":
                return (state[0], state[1] + 1, state[2])
            case "South":
                return (state[0], state[1] - 1, state[2])
            case "East":
                return (state[0] + 1, *state[1:])
            case "West":
                return (state[0] - 1, *state[1:])

In [19]:
testObj = MDPLanding(4)
assert len(testObj.all_actions) == 6
assert [testObj.reward((2,x,0)) for x in range(5)] == [-1,-1,1,-1,-1]
assert [testObj.reward((2,2,x)) for x in range(5)] == [1] + ([testObj.default_reward] * 4)
assert [testObj.result((2,2,2), act) for act in testObj.all_actions] == [
    (2,2,3),
    (2,2,1),
    (2,3,2),
    (2,1,2),
    (3,2,2),
    (1,2,2)
]

#### Part C
Write a function to implement **policy iteration** for this drone landing MDP. Create an MDP environment to represent the $L=4$ case.

Use your function to find an optimal policy for your new MDP environment. Check (by printing to screen) that the policy for the following states are what you expect, and **comment on the results**:
1. $(2,2,1)$
1. $(0,2,1)$
1. $(2,0,1)$

The policy for each of these states is the action that the agent should take in that state. 

In [28]:
def expected_utility(action, state: State, utilities, model: MDPLanding):
    #Utility of doing action in state.
    # Refering to the slides for Policy Iteration Q Learning
    return utilities[model.result(state, action)] # Modified since deterministic

def policy_evaluation(policy, utilities, model: MDPLanding):
    new_utilities = {s:0 for s in model.states}
    for state in model.states:
        reward = model.reward(state)
        spread_reward = model.discount * utilities[model.result(state, policy[state])]
        new_utilities[state] = reward + spread_reward
    return new_utilities

def policy_iteration(model: MDPLanding):
    U = {s:0 for s in model.states}
    policy = {state: np.random.choice(model.actions(state)) for state in model.states} # seed with a random policy

    while True:
        # Utility of states distributed
        U = policy_evaluation(policy, U, model)
        unchanged = True
        for s in model.states:
            # Select the action of possible actions that leads to the square with the most utility
            a = max(model.actions(s), key=lambda a: expected_utility(a, s, U, model))
            if a != policy[s]:
                policy[s] = a
                unchanged = False
        if unchanged:
            return policy

In [30]:
model = MDPLanding(4)
policy = policy_iteration(model)

reference_states = [
    (2,2,1),
    (0,2,1),
    (2,0,1)
]
for state in reference_states:
    print(state, policy[state])

(2, 2, 1) Down
(0, 2, 1) East
(2, 0, 1) North


These policies match what I would expect. The first example is above landing and thus needs to go down. The second example needs to increase it's x value by going east. The third exmple needs to increase its y value by going north.

#### Part D
Provide an example of a non-deterministic transition that could be included in your code in Part C. Describe the function. How would you modify your code to handle a non-deterministic transition function?

We could modify the code such that on any movement there is a 10% chance of also losing height. This modification would happen in the MDPLanding class's function result. I would have to modify part C's Expected_utility and policy_evaluation functions since they each would need to factor in the possibilty of the result not matching expected and would need weighted by the odds that it's different.

#### Part E
Describe the main differences between **policy iteration** and **value iteration**? How would your code change in Part C to convert it to **value iteration**?

Value iteration continually modifies the expected utilities along the graph and watches for when the difference in those values is insignificant. Policy iteration is similar however it uses the values from value iteration and waits until the action to be taken does not change, aka no highest valued neighbors change, it terminates earlier than value iteration.

To change my code in part c I would have to modify the internal loop over states and it's check to see if the policy's action matches the best action to check to see if the expected utility of the tile changed.

#### Part F

Code up a **Q-learning** agent/algorithm to learn how to land the drone. You can do this however you like, as long as you use the MDP class structure defined above.  

Your code should include some kind of a wrapper to run many trials to train the agent and learn the Q values.  You also do not need to have a separate function for the actual "agent"; your code can just be a "for" loop within which you are refining your estimate of the Q values.

From each training trial, save the cumulative discounted reward (utility) over the course of that episode. That is, add up all of $\gamma^t R(s_t)$ where the drone is in state $s_t$ during time step $t$, for the entire sequence. We refer to this as "cumulative reward" because we usually refer to "utility" as the utility *under an optimal policy*.

Some guidelines:
* The drone should initialize in a random non-terminal state for each new training episode.
* The training episodes should be limited to 50 time steps, even if the drone has not yet landed. If the drone lands (in a terminal state), the training episode is over.
* You may use whatever learning rate $\alpha$ you decide is appropriate, and gives good results.
* There are many forms of Q-learning. You can use whatever you would like, subject to the reliability targets below.
* Your code should return:
  * The learned Q values associated with each state-action pair.
  * The cumulative reward for each training trial. 
  * Anything else that might be useful in the ensuing analysis.

In [3]:
# Solution:



#### Part G

Initialize the $L=10$ environment (so that the landing pad is at $(5,5,0)$). Run some number of training trials to train the drone.

**How do I know if my drone is learned enough?**  If you take the mean cumulative reward across the last 5000 training trials, it should be around 0.80. This means at least about 10,000 (but probably more) training episodes will be necessary. It will take a few seconds on your computer, so start small to test your code.

**Then:** Compute block means of cumulative reward from all of your training trials. Use blocks of 500 training trials. This means you need to create some kind of array-like structure such that its first element is the mean of the first 500 trials' cumulative rewards; its second element is the mean of the 501-1000th trials' cumulative rewards; and so on. Make a plot of the block mean rewards as the training progresses. It should increase from about -0.5 initially to somewhere around +0.8.

**And:** Print to the screen the mean of the last 5000 trials' cumulative rewards, to verify that it is indeed about 0.80.

In [1]:
# Solution:


#### Part H

**Question 1:** Why does the cumulative reward start off around -0.5 at the beginning of the training?

**Question 2:** Why will it be difficult for us to train the drone to reliably obtain rewards much greater than about 0.8?

**Your answer here:**

